In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import csv
# a = load_data.load_datas("C:/Users/김세인/Desktop/2020_winter_lab/codes/datas/ml-100k/ml-100k/u.data")
def load_datas(file_name):
    f = open(file_name, 'r')
    lines = f.readlines()
    
    data_set = []
    user_data = []
    item_data = []
    rating_data = []
    for line in lines:
        (user_id, item_id, rating, time_stamp) = line.split('\t')
        some = [int(user_id), int(item_id), float(rating)]
        data_set.append(some)
        user_data.append(int(user_id))
        item_data.append(int(item_id))
        rating_data.append(float(rating))
    return data_set, user_data, item_data, rating_data
"""
make a sets (matrix) of all datas of user id, item id and ratings.
the data_set is composed by [[user, item, rating], [user, item, rating] ... ]
the user_set is composed by [whole of users' id]
the item_set is composed by [whole of items' id]
and n_user and n_item denote the numbers of components of each matrix

for the base cases
""" 
data_set, user_data, item_data, rating_data = load_datas("u1.base")

user_set = np.unique(user_data)
item_set = np.unique(item_data)

n_user = len(user_set)
n_item = len(item_set)


"""
to make matrix of R, we must know the size of test users and items
"""
test_data_set, test_user_data, test_item_data, test_rating_data = load_datas("u1.test")

test_user_set = np.unique(test_user_data)
test_item_set = np.unique(test_item_data)

test_n_user = len(test_user_set)
test_n_item = len(test_item_set)

"""
makes the matrix R, which has the largest value among n_user and test_n_user, and among n_item and test_n_item

to make R starts the index in 1, add the 1 for each column and row, and start to put value at R[1,1]
please notice above be carefully
"""

R = np.zeros((max(user_set[-1], test_user_set[-1])+1,max(item_set[-1],test_item_set[-1])+1))
R_star = np.zeros((max(user_set[-1], test_user_set[-1]),max(item_set[-1],test_item_set[-1])))
for i in range(len(user_data)):
    R[user_data[i],item_data[i]] = rating_data[i]
for j in range(len(user_data)):
    R_star[user_data[j]-1,item_data[j]-1] = rating_data[j]
R_test = np.zeros((max(user_set[-1],test_user_set[-1]),max(item_set[-1],test_item_set[-1])))
for k in range(len(test_user_data)):
    R_test[test_user_data[k]-1,test_item_data[k]-1] = test_rating_data[k]
# print(R_star[0,0])
# print(R[0,0])
# print(R[-1,:])
# print(R[1,1])
# print(R_star.shape)
"""
makes U and V which follow gaussian distribution
do it for 10D and 30D
"""
def initialize_U_V(sigma, sigma_Y, sigma_V, sigma_W, dimension):
    parameter = {}
    Y = sigma_Y*np.random.randn(dimension, user_set[-1])
    V = sigma_V*np.random.randn(dimension, item_set[-1])
    W = sigma_W*np.random.randn(dimension, item_set[-1])
    # print(U.shape)
    # sibal = np.array([U[1,:]])
    # bap = np.array([V[1,:]])
    # # print(U[1,:].shape)
    # # print(U[1:])
    # print(np.dot(sibal.T, bap).shape)

    lamda_Y = (sigma/sigma_Y)**2
    lamda_V = (sigma/sigma_V)**2
    lamda_W = (sigma/sigma_W)**2
    parameter['Y'] = Y
    parameter['V'] = V
    parameter['W'] = W
    parameter['lamda_Y'] = lamda_Y
    parameter['lamda_V'] = lamda_V
    parameter['lamda_W'] = lamda_W
    parameter['sigma'] = sigma
    parameter['sigma_Y'] = sigma_Y
    parameter['sigma_V'] = sigma_V
    parameter['sigma_W'] = sigma_W
    return parameter

def rmse_cal(test_datas, predict_datas):
    l = ~np.isnan(test_datas)
    N = l.sum()
    sqerror = abs(test_datas - predict_datas)**2
    mse = sqerror[l].sum()/N
    return np.sqrt(mse)

def predict(user_id, item_id, parameter, epoch):
    Y= parameter['Y']
    W = parameter['W']
    V = parameter['V']
    lamda_y = parameter['lamda_Y']
    lamda_v = parameter['lamda_V']
    lamda_w = parameter['lamda_W']
    user_vector = np.array(user_data)
    item_vector = np.array(item_data)
    func_out_rmse = []
    testing_results = []
    # func_out_rmse2 = []

    # running_R = np.zeros((max(user_set[-1], test_user_set[-1]),max(item_set[-1],test_item_set[-1])))
    # count=0
    for i in range(epoch):

        shuffled_order = np.arange(user_vector.shape[0])
        np.random.shuffle(shuffled_order)
        final_rmse =[]
        # final_rmse2 = []
        sigma = parameter['sigma']

        "for batch"
        for j in range(10):
            U = np.zeros(Y.shape)

            rmse = []
            # real_rmse = []
            total_count = 0

            testing = np.arange(1000*j, 1000*(j+1))
            indexing = np.mod(testing, shuffled_order.shape[0])

            indexed_user = np.array(user_vector[shuffled_order[indexing]])
            indexed_item = np.array(item_vector[shuffled_order[indexing]])
            for ii in range(indexed_user.shape[0]):
                count_W = 0
                cal_float = np.zeros((10,1))
                for jj in range(indexed_item.shape[0]):
                    if not R[indexed_user[ii],indexed_item[jj]] ==0:
                        #print(np.array([W[:,1]]).T.shape)
                        #print(cal_float.shape)
                        cal_float += np.array([W[:,jj]]).T
                        count_W +=1
                U = Y + cal_float/count_W

            for k in range(10):
                error_rate = 0.0
                numbering = 0
                for ii in range(indexed_user.shape[0]):
                    for jj in range(indexed_item.shape[0]):
                        if not R[indexed_user[ii],indexed_item[jj]] ==0:
                            numbering+=1
                            computing = ((R[indexed_user[ii],indexed_item[jj]]-1)/(5-1) - 1/((1+math.exp(-(U[k,indexed_user[ii]-1]*V[k,indexed_item[jj]-1])))))**2
                            error_rate = (1/2)*computing +(lamda_y/2)*(Y[k,indexed_user[ii]-1])**2 +(lamda_v/2)*(V[k,indexed_item[jj]-1])**2 +(lamda_w/2)*(W[k,indexed_item[jj]-1])**2
                            # error_rate = (1/2)*(R[indexed_user[ii],indexed_item[jj]] - U[k,indexed_user[ii]-1]*V[k,indexed_item[jj]-1])**2 +(lamda_u/2)*(np.linalg.norm(U[k,:]**2) +(lamda_v/2)*(np.linalg.norm(V[k,:])**2)
                rmse.append(error_rate)
                total_count += numbering
                # real_rmse.append((computing/numbering)**0.5)
            # final_rmse.append(max(real_rmse))
            num = rmse.index(min(rmse))
            cal_U = np.array([U[num,:]])
            cal_V = np.array([V[num,:]])
            sample_mean_R = np.dot(cal_U.T, cal_V)
            # print(sample_mean_R)
            sample_R = np.random.normal(sample_mean_R, sigma)
            sample_R[sample_R<1] = 1
            sample_R[sample_R>5] = 5

            # running_R += sample_R
            # count+=1
            # real_running_R = running_R/count
            
            calculated_error = rmse_cal(R_star, sample_R)
            # cal_error = rmse_cal(R_star, real_running_R)
            final_rmse.append(calculated_error)
#             print('observed_rating: %d , RMSE: %f' %(total_count, calculated_error))
            # final_rmse2.append(cal_error)
            # print(final_rmse)


            for kk in range(10):
                for iii in range(indexed_user.shape[0]):
                    Y[kk,indexed_user[iii]-1] = (0.85)*Y[kk,indexed_user[iii]-1] + (0.15)*Y[num,indexed_user[iii]-1]
                for jjj in range(indexed_item.shape[0]):
                    V[kk,indexed_item[jjj]-1] = (0.85)*V[kk,indexed_item[jjj]-1] + (0.15)*V[num,indexed_item[jjj]-1]
                    W[kk,indexed_item[jjj]-1] = (0.85)*W[kk,indexed_item[jjj]-1] + (0.15)*W[num,indexed_item[jjj]-1]
                # print(U[kk,0:5])
                # print(V[kk,0:5])
        array_rmse = np.array(final_rmse)
        # array_rmse2 = np.array(final_rmse2)
        # print(array_rmse)
        out_rmse = np.mean(array_rmse)
        # out_rmse2 = np.mean(array_rmse2)
        print('Training RMSE: %f, at epoch %d' %(out_rmse,i+1))
        # print('Training RMSE_Running: %f, at epoch %d' %(out_rmse2,i+1))
        # f = open('C:/Users/김세인/Desktop/second_try.csv','a',newline='')
        # wr = csv.writer(f)
        # wr.writerow([out_rmse])
        # f.close()

        func_out_rmse.append(out_rmse)
        
        testing_rmse = rmse_cal(R_test, sample_R)
        testing_results.append(testing_rmse)
        print('Testing RMSE: %f, at epoch %d' %(testing_rmse,i+1))
        # func_out_rmse2.append(out_rmse2)

    parameter['W'] = W
    parameter['V'] = V
    parameter['Y'] = Y

    return func_out_rmse,W, V, Y

epochs = 100
dimensions = 10
# para = initialize_U_V(0.03, 0.1,0.1,dimensions)
# para = initialize_U_V(0.0182, 1.82,1.82, dimensions)
para = initialize_U_V(0.08,2.0,2.0,2.0, dimensions)
errors,final_W, final_V, final_Y = predict(None,None,para,epochs)

plt.plot(errors, 'b')
plt.plot(testing_results,'r')
plt.xlabel('epoch')
plt.ylabel('RMSE')
plt.title('PMF')
plt.show()